In [ ]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# Define tools
@tool
def search_web(query: str) -> str:
    """Search the web for information."""
    # Mock implementation
    return f"Search results for '{query}': Found relevant information about the topic."

@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions."""
    try:
        result = eval(expression)  # In production, use a safer math parser
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def weather_tool(location: str) -> str:
    """Get weather information for a location."""
    # Mock implementation
    return f"Weather in {location}: Sunny, 75°F"

# Create tools list
tools = [search_web, calculator, weather_tool]

# Define the state
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

# Initialize LLM with tools
llm = ChatOpenAI(model="gpt-4", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Node functions
def agent_node(state: AgentState):
    """Agent node that can decide to call tools."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def tool_node(state: AgentState):
    """Tool execution node using prebuilt ToolNode."""
    tool_executor = ToolNode(tools)
    return tool_executor.invoke(state)

# Alternative: Custom tool node
def custom_tool_node(state: AgentState):
    """Custom tool execution node."""
    messages = state["messages"]
    last_message = messages[-1]

    # Extract tool calls from the last AI message
    tool_calls = last_message.tool_calls if hasattr(last_message, 'tool_calls') else []

    tool_messages = []
    for tool_call in tool_calls:
        # Find and execute the tool
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]

        # Execute the appropriate tool
        if tool_name == "search_web":
            result = search_web.invoke(tool_args)
        elif tool_name == "calculator":
            result = calculator.invoke(tool_args)
        elif tool_name == "weather_tool":
            result = weather_tool.invoke(tool_args)
        else:
            result = f"Unknown tool: {tool_name}"

        # Create tool message
        tool_message = ToolMessage(
            content=str(result),
            tool_call_id=tool_call["id"]
        )
        tool_messages.append(tool_message)

    return {"messages": tool_messages}

# Edge condition function
def should_continue(state: AgentState):
    """Decide whether to continue to tools or end."""
    messages = state["messages"]
    last_message = messages[-1]

    # If the last message has tool calls, go to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return END

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)  # Using prebuilt ToolNode

# Add edges
workflow.set_entry_point("agent")
workflow.add_conditional_edge(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)
workflow.add_edge("tools", "agent")  # After tools, go back to agent

# Compile the graph
app = workflow.compile()

# Example usage
def run_example():
    """Run an example conversation."""

    # Example 1: Simple question
    print("=== Example 1: Simple Question ===")
    result = app.invoke({
        "messages": [HumanMessage(content="What's the weather like in New York?")]
    })
    print(f"Final response: {result['messages'][-1].content}")

    print("\n=== Example 2: Math Calculation ===")
    result = app.invoke({
        "messages": [HumanMessage(content="Calculate 15 * 7 + 23")]
    })
    print(f"Final response: {result['messages'][-1].content}")

    print("\n=== Example 3: Multiple Tools ===")
    result = app.invoke({
        "messages": [HumanMessage(content="Search for 'LangGraph tutorial' and also calculate 100 / 4")]
    })
    print(f"Final response: {result['messages'][-1].content}")

# Advanced pattern: Different tool strategies per node
class AdvancedAgentState(TypedDict):
    messages: Annotated[List, add_messages]
    current_task: str
    tools_used: List[str]

def research_node(state: AdvancedAgentState):
    """Specialized node for research tasks."""
    research_tools = [search_web]  # Only search tools
    llm_research = llm.bind_tools(research_tools)

    response = llm_research.invoke(state["messages"])
    return {
        "messages": [response],
        "current_task": "research"
    }

def calculation_node(state: AdvancedAgentState):
    """Specialized node for calculations."""
    calc_tools = [calculator]  # Only calculation tools
    llm_calc = llm.bind_tools(calc_tools)

    response = llm_calc.invoke(state["messages"])
    return {
        "messages": [response],
        "current_task": "calculation"
    }

def router_node(state: AdvancedAgentState):
    """Router node to decide which specialized node to use."""
    last_message = state["messages"][-1].content.lower()

    if any(word in last_message for word in ["calculate", "math", "compute"]):
        return "calculation"
    elif any(word in last_message for word in ["search", "find", "research"]):
        return "research"
    else:
        return "general"

# Create advanced workflow
advanced_workflow = StateGraph(AdvancedAgentState)
advanced_workflow.add_node("router", router_node)
advanced_workflow.add_node("research", research_node)
advanced_workflow.add_node("calculation", calculation_node)
advanced_workflow.add_node("general", agent_node)
advanced_workflow.add_node("tools", tool_node)

# Set up routing
advanced_workflow.set_entry_point("router")
advanced_workflow.add_conditional_edge(
    "router",
    lambda x: x.get("current_task", "general"),
    {
        "research": "research",
        "calculation": "calculation",
        "general": "general"
    }
)

# All specialized nodes can go to tools if needed
for node in ["research", "calculation", "general"]:
    advanced_workflow.add_conditional_edge(
        node,
        should_continue,
        {
            "tools": "tools",
            END: END
        }
    )

advanced_workflow.add_edge("tools", END)

if __name__ == "__main__":
    run_example()